In [2]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

In [3]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    #print(json_data)

In [4]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [5]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json()
user_dat = response_json['members']

In [6]:

## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [8]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [9]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


In [10]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'GG API.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [77]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1jmy1jSgzq4gO-P5bKNRdlrVGXcCkdhkhM68aWAKJ9EQ' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 
# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

In [13]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json()
channel_ls = response['channels']

In [14]:

channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01B4PVGLVB',
 'name': 'general',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1600856703,
 'is_archived': False,
 'is_general': True,
 'unlinked': 0,
 'name_normalized': 'general',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01BE2PR6LU',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.',
  'creator': 'U01BE2PR6LU',
  'last_set': 1600856703},
 'previous_names': [],
 'num_members': 63}

In [15]:
#Tìm id của channel #atom-assignment2
for i in channel_ls:
    if i['name'] == 'atom-assignment2':
        atom_assignment2_id = i['id']

print(atom_assignment2_id)

C021FSDN7LJ


In [17]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [19]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [97]:
msg_ls[50]

{'client_msg_id': 'f37184b6-5b1d-4e2e-a0ef-72106846683c',
 'type': 'message',
 'text': '<https://github.com/duongphannamhung/datacracy-atom-duongphannamhung> : Em gửi link assignment 1 của mình. Chúc mọi người nghỉ lễ vui vẻ nha :smiling_face_with_3_hearts:',
 'user': 'U01UMJ5AQ0K',
 'ts': '1618984834.000800',
 'team': 'T01B7SGGMLJ',
 'attachments': [{'service_name': 'GitHub',
   'title': 'duongphannamhung/datacracy-atom-duongphannamhung',
   'title_link': 'https://github.com/duongphannamhung/datacracy-atom-duongphannamhung',
   'text': 'Hôm nay, mình học DATAcracy. Contribute to duongphannamhung/datacracy-atom-duongphannamhung development by creating an account on GitHub.',
   'fallback': 'GitHub: duongphannamhung/datacracy-atom-duongphannamhung',
   'image_url': 'https://opengraph.githubassets.com/42dc9d8f2daadd198aa1736e0a5b85b878bebc00aaf8fd5131f1d4174876aefc/duongphannamhung/datacracy-atom-duongphannamhung',
   'from_url': 'https://github.com/duongphannamhung/datacracy-atom-duongp

In [99]:
range(len(msg_ls))

range(0, 69)

In [92]:

## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
msg_dict = {'latest_reply':[], 'reply_count': [], 'reply_users_count': [], 'reply_users': [], 'github': []  } 

for i in range(len(msg_ls)):
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if user not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: msg_dict['github'].append(github_link[0])
      if 'reply_count' in msg_ls[i].keys(): msg_dict['reply_count'].append(msg_ls[i]['reply_count']) #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): msg_dict['reply_users_count'].append(msg_ls[i]['reply_users_count'])
      if 'reply_users' in msg_ls[i].keys(): msg_dict['reply_users'].append(msg_ls[i]['reply_users'])
      if 'latest_reply' in msg_ls[i].keys(): msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'])))
msg_df = pd.DataFrame(msg_dict)
print(msg_df)

                 latest_reply  reply_count  reply_users_count  \
0  2021-05-15 16:14:31.003600            2                  2   
1  2021-05-18 15:09:59.000800            2                  2   
2  2021-05-08 13:36:43.006000            1                  1   
3  2021-05-11 10:10:06.001300            2                  2   
4  2021-05-15 00:51:26.024600            1                  1   
5  2021-05-15 00:52:35.025100            2                  2   
6  2021-05-11 19:37:14.003100            1                  1   
7  2021-05-14 18:22:06.023600            1                  1   
8  2021-05-11 19:39:13.003700            2                  2   
9  2021-05-10 19:45:15.005800            1                  1   
10 2021-05-15 08:58:19.001900            1                  1   
11 2021-05-11 21:46:48.005500            1                  1   
12 2021-05-11 22:58:18.006600            1                  1   
13 2021-05-13 10:53:49.000600            1                  1   
14 2021-05-14 22:25:02.02

In [91]:
# ACCES GOOGLE SHEET
sheet_index_no1 = 1
spreadsheet_key = '1jmy1jSgzq4gO-P5bKNRdlrVGXcCkdhkhM68aWAKJ9EQ'
sh1 = gc.open_by_key(spreadsheet_key)
worksheet1 = sh1.get_worksheet(sheet_index_no1) #-> 0 - first sheet, 1 - second sheet
# APPEND DATA TO SHEET
set_with_dataframe(worksheet1,msg_df)